# Modeling Component Area, Energy, Latency, and Leak Power

This notebook shows how to integrate component models from the `hwcomponents` library to
model area, energy, latency, and leak power.

Before reading this notebook, you should be familiar with the `hwcomponents` library.
Ensure you've installed it and been through all of the notebooks in the [hwcomponents
documentation](https://accelergy-project.github.io/hwcomponents/).

We'll be using the `component_example.yaml` file, which is annotated explaining what
each attribute means.

In [1]:
from pathlib import Path
from IPython.display import Markdown

examples_dir = Path("../../examples")

display(Markdown(f"""
```yaml
{open(examples_dir / "misc" / "component_annotated.yaml").read()}
```
"""))


```yaml
# The YAML tag at the beginning of the component tells the parser what type of component
# it is.
!Memory

name: GlobalBuffer

# This component_class invokes the hwcomponents_library.SmartBufferSRAM model. The
# component is a SRAM buffer with an address register that is updated on every
# access to queue up the next data.
component_class: SmartBufferSRAM

# Set the size to 16384 bits (64 bits wide, 256 deep)
size: 16384

# Sometimes, hwcomponents models require additional attributes that are not part of the
# accelforge spec. These can be passed to the component models through the
# extra_attributes_for_component_model field.
extra_attributes_for_component_model:
  width: 64
  n_banks: 32
  tech_node: 65e-9

# Actions that the hwcomponents_library model must support. All action attributes that
# can are passed to the model's appropriate method (in this case, `read` or `write`).
# The `bits_per_action` argument is special because it determines how many bits are
# transferred by each read or write. Additionally, to pass extra attributes to the
# component model as arguments, actions also have a extra_attributes_for_component_model
# field.

# For many component models, energy and latency are measured per-bit, so bits_per_action
# may be omitted in these cases.
actions:
- {name: read, bits_per_action: 64}
- {name: write, bits_per_action: 64}

```


Now we'll create a global buffer component and model its area, energy, latency, and leak
power.

In [2]:
# < DOC_INCLUDE_MARKER > single_component_energy_area

# Create the component from the yaml file
import accelforge as af
global_buffer = af.arch.Memory.from_yaml(
    examples_dir / "misc" / "component_annotated.yaml"
)

# Calculate the energy and area of the component
global_buffer = global_buffer.calculate_area_energy_latency_leak()

# Print out the area, leak power, and energy for each action
area = global_buffer.area
leak_power = global_buffer.leak_power
print(f"Area: {area:.2e} m^2")
print(f"Leak power: {leak_power:.2e} W")
for action in global_buffer.actions:
    energy = action.energy
    latency = action.latency
    print(f'Action {action.name} energy {energy:.2e} J, latency {latency:.2e}s')

Area: 2.41e-08 m^2
Leak power: 2.02e-06 W
Action read energy 4.60e-12 J, latency 2.71e-11s
Action write energy 4.92e-12 J, latency 2.71e-11s


Calculating area, energy, latency, and leak power for a component fills out more
attributes as well! We can inspect what is available by looking at the __doc__ or by
looking at this project's documentation website.

In [3]:
print(global_buffer.calculate_area_energy_latency_leak.__doc__)


Calculates the area, energy, latency, and leak power for this component.
Populates the ``area``, ``total_area``, ``leak_power``, ``total_leak_power``,
``total_latency``, and ``component_modeling_log`` fields of this component.
Additionally, for each action, populates the ``<action>.area``,
``<action>.energy``, ``<action>.latency``, and ``<action>.leak_power`` fields.
Extends the ``component_modeling_log`` field with log messages.

Note that these methods will be called by the Spec when calculating energy and
area. If you call them yourself, note that string expressions may not be evaluated
because they need the Spec's global scope. If you are sure that all necessary
values are present and not a result of an expression, you can call these
directly. Otherwise, you can call the
``Spec.calculate_component_area_energy_latency_leak`` and then grab components
from the returned ``Spec``.

Parameters
----------
component_models : list[hwcomponents.ComponentModel] | None
    The models to use f

As we can see in the doc, this function also fills out the `component_modeling_log`
attribute of the component, which contains messages from the component modeling
calculations.

In [4]:
# < DOC_INCLUDE_MARKER > hwcomponents
for message in global_buffer.component_modeling_log:
    print(message)

Initializing SmartBufferSRAM from hwcomponents_library.library.misc
Calling __init__ with arguments {'size': 16384, 'n_banks': 32, 'tech_node': 6.5e-08, 'width': 64}
Unused arguments for ['smart_buffer_sram', 'smartbuffer_sram', 'smartbuffersram'].__init__: (actions, area_scale, bits_per_value_scale, component_class, component_modeling_log, enabled, energy_scale, extra_attributes_for_component_model, latency_scale, leak_power_scale, n_parallel_instances, name, spatial, tensors, total_latency) Arguments used: (size, n_banks, tech_node, width)
Log for subcomponent SRAM:
	Calculated depth: 256.0
	Calling CACTI with cache_size=16384 n_rw_ports=1 block_size=8 tech_node_um=0.065 n_banks=32 tag_size=0 associativity=1
	Calling CACTI with input path /home/jeff/iv4/repos/accelforge/.venv/lib/python3.13/site-packages/hwcomponents_cacti/cacti_inputs_outputs/9d3ab7e8ad44855a9a7a6e2d3086e802edcfc4130060ae0a64a028dd9631dbae
	CACTI output will be written to /home/jeff/iv4/repos/accelforge/.venv/lib/py

To get more details on the specific model used, we can also look at the
`component_model` attribute of the component, which has the
`hwcomponents.ComponentModel` object that was used to calculate the energy and area. We
can use `help()` on this object to get more details on the model.

In [5]:
# < DOC_INCLUDE_MARKER > hwcomponents
component_model = global_buffer.component_model
print(help(component_model)) # Get documentation for the model

Help on SmartBufferSRAM in module hwcomponents_library.library.misc object:

class SmartBufferSRAM(hwcomponents_library.base.LibraryEstimatorClassBase)
 |  SmartBufferSRAM(
 |      tech_node: float,
 |      width: int | None = None,
 |      depth: int | None = None,
 |      size: int | None = None,
 |      n_rw_ports: int = 1,
 |      n_banks: int = 1
 |  )
 |
 |  An SRAM with an address generator that sequentially reads addresses in the SRAM.
 |
 |  Parameters
 |  ----------
 |      tech_node: The technology node in meters. width: The width of the read and write
 |      ports in bits. This is the number of bits
 |          that are accssed by any one read/write. Total size = width * depth.
 |      depth: The number of entries in the SRAM, each with `width` bits. Total size =
 |          width * depth. Either this or size must be provided, but not both.
 |      size: int, optional
 |          The total size of the SRAM in bits. Total size = width * depth. Either this
 |          or dep

We can see that the global buffer is an instance of
`hwcomponents_library.SmartBufferSRAM`, and it contains four subcomponents: an SRAM
buffer, an address register, a delta register, and an addder. We can look at each of
these components to see the area breakdown of our buffer.

In [6]:
# < DOC_INCLUDE_MARKER > hwcomponents

# The model is a hwcomponents_library.SmartBufferSRAM object!
from hwcomponents_library import SmartBufferSRAM

# Type hint it for our IDE
component_model: SmartBufferSRAM = global_buffer.component_model

# Inspect the area of each of its subcomponents
print(f'Area: {component_model.area:.2e} m^2')
print(f'\tSRAM area: {component_model.sram.area:.2e} m^2')
print(f'\tAdder area: {component_model.adder.area:.2e} m^2')
print(f'\tAddress register area: {component_model.address_reg.area:.2e} m^2')
print(f'\tDelta reg area: {component_model.delta_reg.area:.2e} m^2')

Area: 2.41e-08 m^2
	SRAM area: 2.37e-08 m^2
	Adder area: 1.81e-10 m^2
	Address register area: 1.25e-10 m^2
	Delta reg area: 1.25e-10 m^2


Now we'll model the components for a larger design using the `eyeriss`
architecture.

In [7]:
display(Markdown(f"""
```yaml
{open(examples_dir / "arches" / "eyeriss.yaml").read()}
```
"""))


```yaml
arch:
  extra_attributes_for_all_component_models: {tech_node: 65e-9}
  # ============================================================
  # Architecture Description
  # ============================================================
  nodes: # Top-level is hierarchical
  - !Memory # DRAM main memory
    name: MainMemory
    component_class: lpddr4
    size: inf

  - !Memory
    name: GlobalBuffer
    component_class: SmartBufferSRAM
    size: 1024 * 1024 # 1Mb
    # 32 reads and writes per cycle, 200MHz. Note that the bits per read/write is the
    # bits per action set below. This is optional, and if it is omitted, then the total
    # latency is the sum of the latencies of all actions, and action latency is taken
    # from the component models.
    total_latency: (read_actions + write_actions) / 32 / 200e6
    extra_attributes_for_component_model: {n_banks: 32}
    actions:
    - {name: read, bits_per_action: 64}
    - {name: write, bits_per_action: 64}
    tensors: {keep: ~MainMemory, may_keep: All}

  - !Fanout
    name: ArrayFanout
    spatial:
    - {name: reuse_weight, fanout: 14, may_reuse: weight, reuse: weight, min_usage: 1}
    - {name: reuse_output, fanout: 12, may_reuse: output, reuse: output, min_usage: 1}

  - !Memory # Input scratchpad
    name: InputScratchpad
    component_class: SmartBufferSRAM
    size: 12 * 16 # 12 16b entries
    tensors: {keep: input}

  - !Memory # Weight scratchpad
    name: WeightScratchpad
    component_class: SmartBufferSRAM
    size: 192 * 16 # 12 16b entries
    tensors: {keep: weight}

  - !Memory # Output scratchpad
    name: OutputScratchpad
    component_class: SmartBufferSRAM
    size: 16 * 16 # 16 16b entries
    tensors: {keep: output}

  - !Compute # MAC unit
    name: MAC
    component_class: IntMAC
    total_latency: compute_actions / 200e6
    extra_attributes_for_component_model: {multiplier_width: 8, adder_width: 16}

```


The top-level specification object has a `calculate_area_energy_latency_leak` method
that can be used to calculate the area, energy, latency, and leak power of all
components in the design. Additionally, when called from the top-level specification,
this method parses expressions in the architecture, letting you express attributes as
functions of other attributes.

In [8]:
# < DOC_INCLUDE_MARKER > spec_energy_area

# Load the spec and calculate the energy and area of the entire design
spec = af.Spec.from_yaml(examples_dir / "arches" / "eyeriss.yaml")
spec = spec.calculate_component_area_energy_latency_leak()

# Print out the total area and leakage power of the entire design
print(f'Total area of the design: {spec.arch.total_area:.2e} m^2')
print(f'Area breakdown per component:')
for component, area in spec.arch.per_component_total_area.items():
    print(f'\t{component}: {area:.2e} m^2')
print(f'Total leakage power of the design: {spec.arch.total_leak_power:.2e} W')
for component, leak_power in spec.arch.per_component_total_leak_power.items():
    print(f'\t{component}: {leak_power:.2e} W')

WARNING Loading configuration file from /home/jeff/iv4/repos/accelforge/.venv/accelforge/config.yaml


Total area of the design: 7.85e-06 m^2
Area breakdown per component:
	MainMemory: 0.00e+00 m^2
	GlobalBuffer: 6.86e-06 m^2
	InputScratchpad: 7.08e-08 m^2
	WeightScratchpad: 6.02e-07 m^2
	OutputScratchpad: 8.24e-08 m^2
	MAC: 2.35e-07 m^2
Total leakage power of the design: 2.19e-03 W
	MainMemory: 0.00e+00 W
	GlobalBuffer: 2.55e-04 W
	InputScratchpad: 8.08e-05 W
	WeightScratchpad: 1.39e-04 W
	OutputScratchpad: 8.13e-05 W
	MAC: 1.64e-03 W


Once the area, energy, latency, and leak power of the architecture are calculated, we can
look at any individual component and analyze it like we analyzed the global buffer.

In [9]:
mac = spec.arch.find("MAC")
print(f'MAC area: {mac.area:.2e} m^2')
print(f'Total area of all MACs in the architecture: {mac.total_area:.2e} m^2')
print(f'MAC leak power: {mac.leak_power:.2e} W')
print(f'Total leak power of all MACs in the architecture: {mac.total_leak_power:.2e} W')
for action in mac.actions:
    print(f'{action.name} energy: {action.energy:.2e} J, latency {action.latency:.2e}s')

MAC area: 1.40e-09 m^2
Total area of all MACs in the architecture: 2.35e-07 m^2
MAC leak power: 9.73e-06 W
Total leak power of all MACs in the architecture: 1.64e-03 W
compute energy: 2.01e-11 J, latency 0.00e+00s


We can also force the design to use a specific model for a component by passing in our
own `ComponentModel` object. Here we'll create a custom MAC model that has lower
leakage power than the default model.












In [10]:
# Create a hwcomponents model. See the hwcomponents tutorial for ore information on how
# to make these!
import hwcomponents as hwc

class MyMac(hwc.ComponentModel):
    component_name: str = "intmac"
    priority: float = 0.5

    def __init__(self):
        super().__init__(
            area=5e-8,
            leak_power=1e-12 # Very low leakage power
        )

    @hwc.action
    def compute(self) -> float:
        self.logger.info(f'*** Getting compute energy ***')
        # 1pJ, 1ns
        return 1e-12, 1e-9

# Initialize the spec and make the MAC use our custom model
spec = af.Spec.from_yaml(examples_dir / "arches" / "eyeriss.yaml")
mac = spec.arch.find("MAC")
mac.component_model = MyMac()

# Calculate the energy and area of the MAC and print out the results
mac = mac.calculate_area_energy_latency_leak()
print(f'MAC area: {mac.area:.2e} m^2')
print(f'MAC leak power: {mac.leak_power:.2e} W')
for action in mac.actions:
    print(f'{action.name} energy: {action.energy:.2e} J, latency {action.latency:.2e}s')

# The energy and area log contain logs from the ComponentModel component.
print(f"Log messages: ")
for message in mac.component_modeling_log:
    print(f"\t{message}")

WARNING Loading configuration file from /home/jeff/iv4/repos/accelforge/.venv/accelforge/config.yaml


MAC area: 5.00e-08 m^2
MAC leak power: 1.00e-12 W
compute energy: 1.00e-12 J, latency 1.00e-09s
Log messages: 
	Calculating energy for MAC action compute.
	
	Calling action MyMac.compute with arguments () and {}
	*** Getting compute energy ***
	Function compute returned energy 1e-12 and latency 1e-09
	** Final return value for MyMac.compute: energy 1e-12 and latency 1e-09
	Calculating latency for MAC action compute.
	
	Calling action MyMac.compute with arguments () and {}
	*** Getting compute energy ***
	Function compute returned energy 1e-12 and latency 1e-09
	** Final return value for MyMac.compute: energy 1e-12 and latency 1e-09


The `hwcomponents` library has plenty of other component models to use! These can be
listed with the `hwcomponents.get_models` function.

If you're interested in a specific model, use the `help()` function to get more
information on it.

For more information, see the [hwcomponents
documentation](https://accelergy-project.github.io/hwcomponents/).

In [11]:
import hwcomponents as hwc

for model in hwc.get_models()[:5]:
    print(f"{model} supports {model.component_name}")
    for action in model.get_action_names():
        print(f'\t{action}')


from hwcomponents_adc import ADC
help(ADC)

<class 'hwcomponents_adc.main.ADC'> supports ['adc', 'pim_adc', 'sar_adc', 'array_adc', 'pim_array_adc', 'cim_array_adc', 'cim_adc']
	activate
	convert
	drive
	read
	sample
<class 'hwcomponents_neurosim.main.ADC'> supports ['ArrayADC', 'NeuroSimArrayADC']
	add
	compute
	convert
	read
	update
	write
<class 'hwcomponents_neurosim.main.Adder'> supports ['Adder', 'NeuroSimAdder', 'IntAdder']
	add
	compute
	convert
	read
	update
	write
<class 'hwcomponents_neurosim.main.AdderTree'> supports ['AdderTree', 'NeuroSimAdderTree', 'IntAdderTree']
	add
	compute
	convert
	read
	update
	write
<class 'hwcomponents_library.library.aladdin.AladdinAdder'> supports ['Adder', 'AladdinAdder', 'IntAdder']
	add
	read
	write
Help on class ADC in module hwcomponents_adc.main:

class ADC(hwcomponents.model.ComponentModel)
 |  ADC(n_bits: int, tech_node: float, throughput: float, n_adcs=1)
 |
 |  Analog digital converter (ADC) model based on https://arxiv.org/abs/2404.06553.
 |
 |  Args:
 |      n_bits: The numb